In [13]:
import data_process
reload(data_process)

<module 'data_process' from 'data_process.py'>

In [14]:
pre = data_process.pre_process(filedir='/hdd/ctp/day/')

In [15]:
raw_data = pre.loaddata('20180205')

In [16]:
raw_data.shape

(2098929, 9)

In [17]:
ru = raw_data[raw_data['ticker']=='ru1805']

In [7]:
data_dic = pre.trim_merge(raw_data)

date is  20180205
JR803
WH805
RS809
LR803
RI803
fu1803
bb1802
CY805
wr1802
fb1805
PM803


In [8]:
calculated_dic = pre.calc_all_ticker(data_dic, '1s')

/home/hui/anaconda2/lib/python2.7/site-packages/pandas/core/indexing.py:337: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/home/hui/anaconda2/lib/python2.7/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [9]:
calculated_dic['ru1805']

,ticker,bid_price,ask_price,mid_price,rolling_return,aggravated_return
2018-02-05 09:00:00.500,ru1805,12950.0,12955.0,12952.5,0.000000,0.000000
2018-02-05 09:00:01.000,ru1805,12950.0,12955.0,12952.5,0.000000,0.000000
2018-02-05 09:00:01.500,ru1805,12950.0,12955.0,12952.5,0.000000,0.000000
2018-02-05 09:00:02.000,ru1805,12950.0,12955.0,12952.5,0.000000,0.000000
2018-02-05 09:00:02.500,ru1805,12945.0,12950.0,12947.5,-0.000386,-0.000386
2018-02-05 09:00:03.000,ru1805,12945.0,12950.0,12947.5,-0.000386,-0.000386
2018-02-05 09:00:03.500,ru1805,12955.0,12965.0,12960.0,0.000965,0.000579
2018-02-05 09:00:04.000,ru1805,12955.0,12965.0,12960.0,0.000965,0.000579
2018-02-05 09:00:04.500,ru1805,12955.0,12960.0,12957.5,-0.000193,0.000386
2018-02-05 09:00:05.000,ru1805,12955.0,12960.0,12957.5,-0.000193,0.000386


In [18]:
ru

,ticker,bid_price,bid_volume,ask_price,ask_volume,last_price,last_volume,open_interest,turnover
2018-02-05 09:00:00.500,ru1805,12950.0,202,12955.0,1,12950.0,171874,503854,2.236292e+10
2018-02-05 09:00:01.000,ru1805,12950.0,12,12955.0,3,12950.0,172422,503782,2.243389e+10
2018-02-05 09:00:01.500,ru1805,12950.0,1,12955.0,3,12955.0,172632,503804,2.246109e+10
2018-02-05 09:00:02.000,ru1805,12940.0,33,12945.0,70,12940.0,172766,503850,2.247844e+10
2018-02-05 09:00:02.500,ru1805,12945.0,14,12950.0,36,12945.0,173386,503880,2.255871e+10
2018-02-05 09:00:03.000,ru1805,12950.0,4,12955.0,12,12950.0,173692,503906,2.259833e+10
2018-02-05 09:00:03.500,ru1805,12955.0,6,12965.0,14,12955.0,173788,503916,2.261076e+10
2018-02-05 09:00:04.000,ru1805,12955.0,17,12960.0,2,12955.0,173962,503902,2.263331e+10
2018-02-05 09:00:04.500,ru1805,12955.0,1,12960.0,4,12960.0,174098,503908,2.265093e+10
2018-02-05 09:00:05.000,ru1805,12950.0,88,12955.0,1,12950.0,174216,503866,2.266622e+10


In [6]:
ru = raw_data[raw_data['ticker']=='ru1805']
ru[['bid_price', 'ask_price']]

,bid_price,ask_price
2018-02-05 09:00:00.500,12950.0,12955.0
2018-02-05 09:00:01.500,12950.0,12955.0
2018-02-05 09:00:01.500,12950.0,12955.0
2018-02-05 09:00:02.500,12945.0,12950.0
2018-02-05 09:00:02.500,12940.0,12945.0
2018-02-05 09:00:03.500,12955.0,12965.0
2018-02-05 09:00:03.500,12950.0,12955.0
2018-02-05 09:00:04.500,12955.0,12960.0
2018-02-05 09:00:04.500,12955.0,12960.0
2018-02-05 09:00:05.500,12955.0,12965.0


In [100]:
data_dic = pre.trim_merge(raw_data)

date is  20180403
JR807
WH805
RS807
RI811
fu1805
bb1805
CY807
wr1804
fb1805
PM805


In [12]:
ap = data_dic['AP805']

In [15]:
ap.fillna(method='ffill', inplace=True)

In [26]:
tmp = pre.calcAll(ap, period='0s')

In [41]:
calculated_dic = pre.calc_all_ticker(data_dic, '500ms')

new_sample_lib.py:49: RuntimeWarning: invalid value encountered in double_scalars
  ret = (morning_mid[i] - morning_mid[i-step])/morning_mid[i-step]
new_sample_lib.py:58: RuntimeWarning: invalid value encountered in double_scalars
  ret = (afternoon_mid[i] - afternoon_mid[i - step]) / afternoon_mid[i - step]
new_sample_lib.py:58: RuntimeWarning: divide by zero encountered in double_scalars
  ret = (afternoon_mid[i] - afternoon_mid[i - step]) / afternoon_mid[i - step]
new_sample_lib.py:49: RuntimeWarning: divide by zero encountered in double_scalars
  ret = (morning_mid[i] - morning_mid[i-step])/morning_mid[i-step]


In [44]:
calculated_dic['ru1809']['rolling_return'].values

array([ 0.        ,  0.        , -0.00086749, ...,  0.        ,
        0.        ,  0.        ])

In [50]:
return_df = pre.merge_return(calculated_dic)

In [55]:
daylst = pre.generateDayLst(20180319,20180320)

In [82]:
return_dic = pre.load_multi_days(['20180409'], period='0s')

date is  20180409


In [75]:
return_dic['20180319'].shape

(28798, 41)

In [88]:
ru = data_dic['ru1809']

In [91]:
count_dic = {}
for key in data_dic.keys():
    tmp = data_dic[key]
    mid = tmp['bid_price']+tmp['ask_price']
    count = 0
    for i in mid:
        if i == 0:
            count += 1
    count_dic[key] = count

In [103]:
ru['ask_price'].shift(-500, freq='ms')

2018-04-03 09:00:00.000    11530.0
2018-04-03 09:00:00.500    11530.0
2018-04-03 09:00:01.000    11525.0
2018-04-03 09:00:01.500    11525.0
2018-04-03 09:00:02.000    11505.0
2018-04-03 09:00:02.500    11505.0
2018-04-03 09:00:03.000    11515.0
2018-04-03 09:00:03.500    11515.0
2018-04-03 09:00:04.000    11510.0
2018-04-03 09:00:04.500    11510.0
2018-04-03 09:00:05.000    11515.0
2018-04-03 09:00:05.500    11515.0
2018-04-03 09:00:06.000    11520.0
2018-04-03 09:00:06.500    11520.0
2018-04-03 09:00:07.000    11520.0
2018-04-03 09:00:07.500    11520.0
2018-04-03 09:00:08.000    11520.0
2018-04-03 09:00:08.500    11520.0
2018-04-03 09:00:09.000    11525.0
2018-04-03 09:00:09.500    11525.0
2018-04-03 09:00:10.000    11530.0
2018-04-03 09:00:10.500    11530.0
2018-04-03 09:00:11.000    11525.0
2018-04-03 09:00:11.500    11525.0
2018-04-03 09:00:12.000    11520.0
2018-04-03 09:00:12.500    11520.0
2018-04-03 09:00:13.000    11520.0
2018-04-03 09:00:13.500    11520.0
2018-04-03 09:00:14.

In [94]:
raw_data[raw_data['ticker'] == 'wr1804']

,ticker,bid_price,bid_volume,ask_price,ask_volume,last_price,last_volume,open_interest,turnover
2018-04-03 09:00:00.500,wr1804,0.0,0,0.0,0,3121.0,0,0,0.0
2018-04-03 10:15:00.500,wr1804,0.0,0,0.0,0,3121.0,0,0,0.0
2018-04-03 10:30:00.500,wr1804,0.0,0,0.0,0,3121.0,0,0,0.0
2018-04-03 13:30:00.500,wr1804,0.0,0,0.0,0,3121.0,0,0,0.0


In [108]:
data = return_dic['20180409']

In [110]:
tmp = data['ru0']

tmp

In [114]:
tmp = tmp.values[2:]

In [118]:
data.replace()

ValueError: Length of values does not match length of index

In [117]:
tmp

array([0.00131637, 0.        , 0.00065732, ..., 0.        , 0.        ,
       0.        ])